In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

이전 논문에서는 인코더에 들어가는 문장들을 역으로 뒤집었지만 이번에는 그렇게하진않는다.

In [25]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

데이터를 처리하는 필드를 만들며 , 소문자로 통일하면서 eos와 sos를 추가한다.

In [27]:
SRC = Field(tokenize=tokenize_de, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

TRG = Field(tokenize = tokenize_en, 
            init_token='<sos>',  
            eos_token='<eos>', 
            lower=True)

In [28]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

In [32]:
print(vars(train_data.examples[0]))

{'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [33]:
SRC.build_vocab(train_data, min_freq =2 )
TRG.build_vocab(train_data, min_freq =2 )

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

# Building the seq2seq Model

## Encoder

인코더는 기존의 LSTM이 GRU로 바뀐 것을 제외하면 비슷하다. 또한 RNN 멀티레이어들 간에 적용되는 드롭아웃이 하나의 레이어로 바뀌면서 쓰지않는다. 

GRU의 또다른 특징은 기존에 cell state까지 인풋으로 넣고 반환했던 LSTM과 달리 전 시점의 은닉상태와 $x_t$만 들어가고 반환한다는 것이다.
$$h_t = GRU(x_t,h_{t-1})$$

RNN과 GRU의 수식은 동일해보이지만 GRU 내부에는 은닉상태의 정보의 흐름을 통제하는 많은 게이트 매커니즘이 존재한다. 그 밖에 시퀀스를 넣어서 context vector인 $z=h_T$ 반환하는 것 이외에는 같다.

In [36]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #드롭아웃없음
        
        self.rnn = nn.GRU(emb_dim,hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #cell state없음
        
        return hidden

# Decoder

디코더는 기존모델과 실행부분에서 다소 차이가 있으며 정보에 대한 압박을 덜어준다.(LSTM과 비교해서) 

GRU에서는 단지 타겟 토큰인 $y_t$와 기존 은닉상태이던 $s_{t-1}$만 인풋으로 넣는 것 대신에 추가로 맥락 벡터인 $z$를 추가한다.
$$s_t = DecoderGRU(y_t,s_{t-1},z)$$

이 맥락벡터인 $z$는 인코더에서 최종 반환되어 디코더의 모든 매 시점에서 재사용되게 된다.
이전 모델에서 선형 함수 $f$에 디코더의 탑 레이어의 최종 은닉 상태 $s_t$를 이용해 그 다음 토큰인 $y_{t+1}$을 예측했다면 이제는, 현재 토큰과 맥락 벡터 $z$를 선형함수에 넣어서 다음 토큰을 예측한다.
$$\hat{y}_{t+1} = f(y_t, s_t, z)$$

여기서 주의할 점은 초기의 은닉상태 $s_t$는 여전히 맥락벡터인 $z_t$와 같으므로 처음 토|큰을 생성할 때 사실상 두개의 동일한 맥락 벡터를 GRU에 투입하는 것과 같다.

어떻게 이 두가지 변화가 정보의 압박을 줄였냐면 이론적으로 디코더의 은닉상태인 $s_t$가 항상 인풋을 넣으면 되는 소스 문장의 정보를 더이상 담을 필요가 없기 때문이다. 그러므로 지금까지 토큰들이 어떤 정보를 생성했는지만 알면 된다. 추가적으로 선형 함수에 들어가는 $y_t$는 은닉상태에서 정보를 가져올 필요없이 이 레이어에서 직접적으로 어떤 토큰을 살필 것인지를 의미한다.

가설은 가설이기때문에 단정적으로 모델이 주입된 정보를 어떻게 처리하는지에 대해서 말할 수 는 없지만 이 아이디어는 확실히 직관적이고 결과 또한 그렇게 보인다.

In [37]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #sent len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
         
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden 

# Seq2Seq Model

인코더와 디코더의 은닉 차원은 동일해야하며 다음것들도 마찬가지이다:

원 문장인 X는 인코더로 들어가서 맥락 벡터를 산출한다.
초기의 디코더의 은닉 상태는 맥락벡터로 정해져있다.<br/>
$y_1$의 첫번째 인풋으로는 <sos>토큰을 사용한다.<br/>
다음 루프문을 디코드한다<br/>
    
    
    디코더에 $y_t, s_{t-1},z$를 집어넣는다.
    예측값인 $\hat{y}_{t+1}$과 새로운 은닉상태인 $s_t$를 얻는다.
    다음인풋값을 적절하게 세팅함으로써 강사 강요를 할것인지 아닌지 결정한다

In [39]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, max_len):
            
            output, hidden = self.decoder(input, hidden, context)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)

        return outputs
    

# Training the Seq2Seq Model

In [43]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)


매개변수를 설정한다. 논문에서는 매개변수가 정규분포형태의 평균은 0이고 표준편차는 0.01을 따른다고 한다. 또한 recurrent parameter에 대해서는 special initialization을 해야한다고 하지만 앞과 같게 두겠다.

In [45]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std = 0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(768, 512)
    (out): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5)
  )
)

비록 인코더와 디코더에 단일의 선형 RNN층을 가지고 있지만 , 실제로는 저번 모델보다 매개변수 수가 많다. GRU와 선형 레이어에 input의 사이즈가 커졌기 때문이다. 그렇지만 수가 그렇게 많진 않고 연산하는데있어서 에폭당 3초정도의 훈련시간을 늘릴 뿐이다.

In [46]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model ahs {count_parameters(model):,} trainable parameters')

The model ahs 14,219,781 trainable parameters


In [47]:
optimizer = optim.Adam(model.parameters())

또한 손실함수를 만들어 패드 토큰에 대한 손실을 무시하게 해야한다.

In [48]:
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [49]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [50]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [51]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [52]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 24s
	Train Loss: 5.081 | Train PPL: 160.896
	 Val. Loss: 5.075 |  Val. PPL: 159.939
Epoch: 02 | Time: 0m 25s
	Train Loss: 4.468 | Train PPL:  87.183
	 Val. Loss: 5.479 |  Val. PPL: 239.632
Epoch: 03 | Time: 0m 25s
	Train Loss: 4.198 | Train PPL:  66.522
	 Val. Loss: 5.081 |  Val. PPL: 160.892
Epoch: 04 | Time: 0m 26s
	Train Loss: 3.881 | Train PPL:  48.459
	 Val. Loss: 4.574 |  Val. PPL:  96.900
Epoch: 05 | Time: 0m 26s
	Train Loss: 3.561 | Train PPL:  35.215
	 Val. Loss: 4.348 |  Val. PPL:  77.351
Epoch: 06 | Time: 0m 26s
	Train Loss: 3.275 | Train PPL:  26.436
	 Val. Loss: 4.079 |  Val. PPL:  59.088
Epoch: 07 | Time: 0m 25s
	Train Loss: 3.034 | Train PPL:  20.780
	 Val. Loss: 3.932 |  Val. PPL:  51.024
Epoch: 08 | Time: 0m 25s
	Train Loss: 2.796 | Train PPL:  16.385
	 Val. Loss: 3.810 |  Val. PPL:  45.143
Epoch: 09 | Time: 0m 25s
	Train Loss: 2.582 | Train PPL:  13.228
	 Val. Loss: 3.776 |  Val. PPL:  43.638
Epoch: 10 | Time: 0m 25s
	Train Loss: 2.427 | Train PPL

In [53]:
import numpy as np

In [57]:
a = np.arange(10)
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [60]:
a[5:8]=100

In [61]:
a

array([  0,   1,   2,   3,   4, 100, 100, 100,   8,   9])

In [62]:
li= list(range(10))

In [86]:
a=float(input())
print('%.6f'%a)

0.14
0.140000
